In [26]:
import torch
import pandas as pd
from torch_geometric.nn import HeteroConv, Linear, SAGEConv, to_hetero
import torch.nn.functional as F
import torch_geometric.transforms as T



In [2]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [16]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [14]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [17]:
customer_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_orders","n_not_returned","n_partial_returned","n_fully_returned","n_products"]}
order_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["label","n_products"]}
product_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_orders", "n_returned"]}
customer_x, customer_mapping = load_node_csv('data/customer_node_attributes.csv', index_col='customer', encoders=customer_encoders)
order_x, order_mapping = load_node_csv('data/order_node_attributes.csv', index_col='order', encoders=order_encoders)
product_x, product_mapping = load_node_csv('data/product_node_attributes.csv', index_col='product', encoders=product_encoders)

In [18]:
customer_encoders

{'n_orders': <__main__.IdentityEncoder at 0x18848e5a490>,
 'n_not_returned': <__main__.IdentityEncoder at 0x18848e73010>,
 'n_partial_returned': <__main__.IdentityEncoder at 0x18848e71ad0>,
 'n_fully_returned': <__main__.IdentityEncoder at 0x18848e73610>,
 'n_products': <__main__.IdentityEncoder at 0x18848e72a10>}

In [19]:
customer_order_edge_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_products","label","train_masks","valid_masks","test_masks"]}
customer_order_edge_index, customer_order_edge_attr = load_edge_csv('data/customer_edgelist.csv', 'customer', customer_mapping, 'order', order_mapping, encoders=customer_order_edge_encoders)
order_product_edge_index, order_product_edge_attr = load_edge_csv('data/order_edgelist.csv', 'order', order_mapping, 'product', product_mapping, encoders={"label": IdentityEncoder(torch.long)})

In [52]:
from torch_geometric.data import HeteroData

data = HeteroData()

customer_mapping = []
order_mapping = []
item_mapping = []


# data['customer'].x = customer_x 
data['order'].x = order_x
data['product'].x = product_x
data['customer', 'order'].edge_index = customer_order_edge_index
data['customer', 'order'].edge_attr = customer_order_edge_attr[:,:-1]
data['customer', 'order'].edge_label = customer_order_edge_attr[:,-1]

data['order',  'product'].edge_index = order_product_edge_index
data['order',  'product'].edge_attr = order_product_edge_attr[:,:-1]
data['order',  'product'].edge_label = order_product_edge_attr[:,-1]

print(data)

HeteroData(
  order={ x=[849185, 2] },
  product={ x=[58415, 2] },
  (customer, to, order)={
    edge_index=[2, 849185],
    edge_attr=[849185, 4],
    edge_label=[849185],
  },
  (order, to, product)={
    edge_index=[2, 2666262],
    edge_attr=[2666262, 0],
    edge_label=[2666262],
  }
)


In [53]:
# DOES NOT WORK FROM HERE

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
    
data = T.ToUndirected()(data)

model = GNN(hidden_channels=64, out_channels=3)
model = to_hetero(model, data.metadata(), aggr='sum')

In [58]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    pred = model(data.x_dict, data.edge_index,
                 )
    target = data['customer', 'order'].edge_label
    loss = torch.nn.CrossEntropyLoss(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)

In [59]:
for epoch in range(1, 5):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

AttributeError: 'HeteroData' has no attribute 'edge_index'